In [51]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.utils import resample
sns.set()

In [52]:
# Carrega dataset
gastos_diretos = pd.read_csv('../databases/gastosdiretos.csv', encoding='utf-8')

# Testes de Hipótese

## Teste: Gastos realizados entre diferentes categorias econômicas
Hipótese Nula (H0): As médias dos gastos realizados são iguais entre todas as categorias econômicas.

Hipótese Alternativa (H1): Pelo menos uma das médias dos gastos realizados é diferente entre as categorias econômicas.

In [53]:
gastos_por_categoria = gastos_diretos.groupby('NOME CATEGORIA ECONÔMICA')['ORÇAMENTO REALIZADO (R$)'].apply(list)

# Teste ANOVA
f_statistic, p_value = stats.f_oneway(*gastos_por_categoria)

print(f'Estatística F: {f_statistic:.4f}')
print(f'Valor p: {p_value:.4f}')

if p_value < 0.05:
    print("Rejeitamos a hipótese nula. As categorias econômicas afetam significativamente os gastos realizados.")
else:
    print("Não rejeitamos a hipótese nula. As categorias econômicas não afetam significativamente os gastos realizados.")

Estatística F: 2.7861
Valor p: 0.0250
Rejeitamos a hipótese nula. As categorias econômicas afetam significativamente os gastos realizados.


## Teste: Gastos realizados entre diferentes funções.

Hipótese Nula (H0): As médias dos gastos realizados são iguais entre todas as funções.

Hipótese Alternativa (H1): Pelo menos uma das médias dos gastos realizados é diferente entre as 

In [54]:
gastos_por_funcao = gastos_diretos.groupby('NOME FUNÇÃO')['ORÇAMENTO REALIZADO (R$)'].apply(list)

f_statistic, p_value = stats.f_oneway(*gastos_por_funcao)

print(f'Estatística F: {f_statistic:.4f}')
print(f'Valor p: {p_value:.4f}')

if p_value < 0.05:
    print("Rejeitamos a hipótese nula. As funções afetam significativamente os gastos realizados.")
else:
    print("Não rejeitamos a hipótese nula. As funções não afetam significativamente os gastos realizados.")

Estatística F: 12.3393
Valor p: 0.0000
Rejeitamos a hipótese nula. As funções afetam significativamente os gastos realizados.


## Teste: Gastos realizados entre diferentes órgãos superiores.

Hipótese Nula (H0): As médias dos gastos realizados são iguais entre todos os órgãos superiores.

Hipótese Alternativa (H1): Pelo menos uma das médias dos gastos realizados é diferente entre os órgãos superiores.

In [55]:
gastos_por_orgao_superior = gastos_diretos.groupby('NOME ÓRGÃO SUPERIOR')['ORÇAMENTO REALIZADO (R$)'].apply(list)

f_statistic, p_value = stats.f_oneway(*gastos_por_orgao_superior)

print(f'Estatística F: {f_statistic:.4f}')
print(f'Valor p: {p_value:.4f}')

if p_value < 0.05:
    print("Rejeitamos a hipótese nula. Os órgãos superiores afetam significativamente os gastos realizados.")
else:
    print("Não rejeitamos a hipótese nula. Os órgãos superiores não afetam significativamente os gastos realizados.")

Estatística F: 36.6781
Valor p: 0.0000
Rejeitamos a hipótese nula. Os órgãos superiores afetam significativamente os gastos realizados.


# Intervalos de Confiança

In [56]:
def ic(df, column):
    data = df[column]
    mean = data.mean()
    std = data.std(ddof=1)
    se = std / np.sqrt(len(data))
    return (mean - 1.96 * se, mean + 1.96 * se)

ic_categoria1 = ic(gastos_diretos[gastos_diretos['NOME CATEGORIA ECONÔMICA'] == 'DESPESAS CORRENTES'], 'ORÇAMENTO REALIZADO (R$)')
ic_categoria2 = ic(gastos_diretos[gastos_diretos['NOME CATEGORIA ECONÔMICA'] == 'DESPESAS DE CAPITAL'], 'ORÇAMENTO REALIZADO (R$)')

# Os gastos realizados estimados com 95% de confiança
print(f'Intervalo de Confiança para DESPESAS CORRENTES: {ic_categoria1}')
print(f'Intervalo de Confiança para DESPESAS DE CAPITAL: {ic_categoria2}')

Intervalo de Confiança para DESPESAS CORRENTES: (3861676771.7773914, 6611853948.729307)
Intervalo de Confiança para DESPESAS DE CAPITAL: (3811194527.0582542, 20658402745.316418)


# Intervalos de Confiança MSE

In [57]:
X = gastos_diretos[['ORÇAMENTO INICIAL (R$)', 'Year', 'NOME CATEGORIA ECONÔMICA']]
y = gastos_diretos['ORÇAMENTO ATUALIZADO (R$)']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['ORÇAMENTO INICIAL (R$)', 'Year']),  # Normaliza variáveis numéricas
        ('cat', OneHotEncoder(), ['NOME CATEGORIA ECONÔMICA'])   # Codifica variáveis categóricas
    ])

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

variancia = np.var(y)
mse_normalizado_variancia = mse / variancia
nmse = mse / variancia

print(f'MSE: {mse:.2f}')
print(f'R²: {r2:.2f}')
print(f'MSE Normalizado pela Variância: {mse_normalizado_variancia:.2e}')

MSE: 3666237674105327845376.00
R²: 0.90
MSE Normalizado pela Variância: 1.21e-01


In [59]:
def bootstrap_mse_ci(model, X_test, y_test, n_iterations=1000, ci=95):
    mses = []
    n_size = len(X_test)

    for _ in range(n_iterations):
        X_resampled, y_resampled = resample(X_test, y_test, n_samples=n_size, random_state=None)

        y_pred_resampled = model.predict(X_resampled)
        
        mse = mean_squared_error(y_resampled, y_pred_resampled)
        mses.append(mse)

    lower = np.percentile(mses, (100 - ci) / 2)
    upper = np.percentile(mses, 100 - (100 - ci) / 2)
    
    return lower, upper

lower_ci, upper_ci = bootstrap_mse_ci(model, X_test, y_test)
print(f'Intervalo de Confiança para MSE: ({lower_ci:.2f}, {upper_ci:.2f})')

Intervalo de Confiança para MSE: (1076355243226351140864.00, 7887813411064541872128.00)


O alto valor do MSE pode levantar preocupações, posto isso, normalizamos. 
O valor do R² demonstra que o modelo é eficaz na explicação da variabilidade dos dados.